# Train Language Model on Coq Proofs (Carte Blanche)

In [38]:
import torch
from transformers import AutoModel, AutoModelForSeq2SeqLM, AutoConfig
from pathlib import Path
from glob import glob

### Train Byte Level Encoder
Requires special processing to remove the extended Unicode characters present in some of these files

E.g. ``->, \/, /\, etc``

In [48]:
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.models import BPE
from tokenizers.normalizers import Lowercase, NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer

Create Coq file paths.

In [136]:
paths = [x for x in Path('..').glob("coq_projects/**/*.v")]

Just rewrite all the paths to utf-8

In [154]:
DATA_PATH = Path("txt/")
for p in paths:
    text = p.open('rb').read().decode('utf-8', 'ignore')
    p = '/'.join(p.parts[1:])
    p = DATA_PATH / p
    p.parent.mkdir(parents=True, exist_ok=True)
    print(p)
    with p.open('w', encoding='utf-8') as f:
        f.write(text)

txt/coq_projects/ruler-compass-geometry/E1_Incidence.v
txt/coq_projects/ruler-compass-geometry/C15_Parallelogramm.v
txt/coq_projects/ruler-compass-geometry/E5_Parallelisme.v
txt/coq_projects/ruler-compass-geometry/B3_Alignes_Prop.v
txt/coq_projects/ruler-compass-geometry/B8_Point_Def.v
txt/coq_projects/ruler-compass-geometry/C9_Triangles_Emboites.v
txt/coq_projects/ruler-compass-geometry/A2_Orientation.v
txt/coq_projects/ruler-compass-geometry/C14_Angle_Droit.v
txt/coq_projects/ruler-compass-geometry/B2_Orient_Prop.v
txt/coq_projects/ruler-compass-geometry/B5_Entre_Prel.v
txt/coq_projects/ruler-compass-geometry/C10_Milieu.v
txt/coq_projects/ruler-compass-geometry/E3_Congruence.v
txt/coq_projects/ruler-compass-geometry/B11_Angle_prop.v
txt/coq_projects/ruler-compass-geometry/D6_SumAnglesProp.v
txt/coq_projects/ruler-compass-geometry/C4_Triangles_non_degeneres_egaux.v
txt/coq_projects/ruler-compass-geometry/C2_Entre_Prop.v
txt/coq_projects/ruler-compass-geometry/B9_Inegalite_Triang.v
txt

txt/coq_projects/concat/CATEGORY_THEORY/LIMITS/FunForget_UA.v
txt/coq_projects/concat/CATEGORY_THEORY/LIMITS/Iso_Limit.v
txt/coq_projects/concat/CATEGORY_THEORY/LIMITS/Limit.v
txt/coq_projects/concat/CATEGORY_THEORY/LIMITS/PULB.v
txt/coq_projects/concat/CATEGORY_THEORY/CATEGORY/Hom_Equality.v
txt/coq_projects/concat/CATEGORY_THEORY/CATEGORY/Category.v
txt/coq_projects/concat/CATEGORY_THEORY/CATEGORY/ONE.v
txt/coq_projects/concat/CATEGORY_THEORY/CATEGORY/MON.v
txt/coq_projects/concat/CATEGORY_THEORY/CATEGORY/Dual.v
txt/coq_projects/concat/CATEGORY_THEORY/CATEGORY/PROD.v
txt/coq_projects/concat/CATEGORY_THEORY/CATEGORY/PermCat.v
txt/coq_projects/concat/CATEGORY_THEORY/CATEGORY/SET.v
txt/coq_projects/concat/CATEGORY_THEORY/CATEGORY/FullSubCat.v
txt/coq_projects/concat/CATEGORY_THEORY/CATEGORY/CONSTRUCTIONS/SET_Exponents.v
txt/coq_projects/concat/CATEGORY_THEORY/CATEGORY/CONSTRUCTIONS/SET_Pullback.v
txt/coq_projects/concat/CATEGORY_THEORY/CATEGORY/CONSTRUCTIONS/CCC.v
txt/coq_projects/conca

txt/coq_projects/math-comp/mathcomp/solvable/pgroup.v
txt/coq_projects/math-comp/mathcomp/solvable/abelian.v
txt/coq_projects/math-comp/mathcomp/solvable/burnside_app.v
txt/coq_projects/math-comp/mathcomp/solvable/extremal.v
txt/coq_projects/math-comp/mathcomp/solvable/nilpotent.v
txt/coq_projects/math-comp/mathcomp/solvable/cyclic.v
txt/coq_projects/math-comp/mathcomp/solvable/maximal.v
txt/coq_projects/math-comp/mathcomp/solvable/finmodule.v
txt/coq_projects/math-comp/mathcomp/solvable/hall.v
txt/coq_projects/math-comp/mathcomp/solvable/alt.v
txt/coq_projects/math-comp/mathcomp/solvable/gfunctor.v
txt/coq_projects/math-comp/mathcomp/solvable/sylow.v
txt/coq_projects/math-comp/mathcomp/solvable/all_solvable.v
txt/coq_projects/math-comp/mathcomp/solvable/commutator.v
txt/coq_projects/math-comp/mathcomp/solvable/extraspecial.v
txt/coq_projects/math-comp/mathcomp/solvable/jordanholder.v
txt/coq_projects/math-comp/mathcomp/solvable/center.v
txt/coq_projects/math-comp/mathcomp/solvable/gse

txt/coq_projects/disel/Core/InjectionOld.v
txt/coq_projects/disel/Core/Always.v
txt/coq_projects/disel/Core/DiSeLExtraction.v
txt/coq_projects/disel/Core/DepMaps.v
txt/coq_projects/disel/Core/Rely.v
txt/coq_projects/disel/Core/StatePredicates.v
txt/coq_projects/disel/Core/NewStatePredicates.v
txt/coq_projects/disel/Examples/SeqLib.v
txt/coq_projects/disel/Examples/Greeter/Greeter.v
txt/coq_projects/disel/Examples/Querying/QueryPlusTPC.v
txt/coq_projects/disel/Examples/Querying/QueryProtocol.v
txt/coq_projects/disel/Examples/Querying/QueryHooked.v
txt/coq_projects/disel/Examples/LockResource/ResourceProtocol.v
txt/coq_projects/disel/Examples/LockResource/LockProtocol.v
txt/coq_projects/disel/Examples/TwoPhaseCommit/TwoPhaseParticipant.v
txt/coq_projects/disel/Examples/TwoPhaseCommit/TwoPhaseInductiveProof.v
txt/coq_projects/disel/Examples/TwoPhaseCommit/SimpleTPCApp.v
txt/coq_projects/disel/Examples/TwoPhaseCommit/TwoPhaseProtocol.v
txt/coq_projects/disel/Examples/TwoPhaseCommit/TwoPhas

txt/coq_projects/group-theory/gr.v
txt/coq_projects/group-theory/g2.v
txt/coq_projects/group-theory/g1.v
txt/coq_projects/group-theory/Z/Z_succ_pred.v
txt/coq_projects/group-theory/Z/Zle.v
txt/coq_projects/group-theory/Z/Zadd.v
txt/coq_projects/group-theory/Z/Zbase.v
txt/coq_projects/group-theory/Z/Nat_complements.v
txt/coq_projects/hardware/Factorization/Factorization_Verif.v
txt/coq_projects/hardware/Factorization/Factorization_Synth.v
txt/coq_projects/hardware/Factorization/Factorization.v
txt/coq_projects/hardware/Factorization/Factorization_Prog.v
txt/coq_projects/hardware/Factorization/Linear_Structures.v
txt/coq_projects/hardware/Factorization/Comparator/Comp_Verif.v
txt/coq_projects/hardware/Factorization/Comparator/Comparator_Relation.v
txt/coq_projects/hardware/Factorization/Comparator/Comp_Simpl.v
txt/coq_projects/hardware/Factorization/Comparator/Comp_Synth.v
txt/coq_projects/hardware/Factorization/Comparator/extract.v
txt/coq_projects/hardware/Factorization/Comparator/Comp

txt/coq_projects/GeoCoq/Meta_theory/Parallel_postulates/alternate_interior_angles_consecutive_interior_angles.v
txt/coq_projects/GeoCoq/Meta_theory/Parallel_postulates/weak_inverse_projection_postulate_bachmann_s_lotschnittaxiom.v
txt/coq_projects/GeoCoq/Meta_theory/Parallel_postulates/weak_tarski_s_parallel_postulate_weak_inverse_projection_postulate.v
txt/coq_projects/GeoCoq/Meta_theory/Parallel_postulates/bachmann_s_lotschnittaxiom_weak_inverse_projection_postulate.v
txt/coq_projects/GeoCoq/Meta_theory/Parallel_postulates/playfair_par_trans.v
txt/coq_projects/GeoCoq/Meta_theory/Parallel_postulates/universal_posidonius_postulate_par_perp_perp.v
txt/coq_projects/GeoCoq/Meta_theory/Parallel_postulates/bachmann_s_lotschnittaxiom_weak_triangle_circumscription_principle.v
txt/coq_projects/GeoCoq/Meta_theory/Parallel_postulates/rah_thales_postulate.v
txt/coq_projects/GeoCoq/Meta_theory/Parallel_postulates/SPP_ID.v
txt/coq_projects/GeoCoq/Meta_theory/Parallel_postulates/playfair_existential

txt/coq_projects/distributed-reference-counting/machine2/invariant4.v
txt/coq_projects/distributed-reference-counting/machine2/machine.v
txt/coq_projects/distributed-reference-counting/machine2/alternate.v
txt/coq_projects/distributed-reference-counting/machine2/invariant0.v
txt/coq_projects/distributed-reference-counting/machine2/invariant7.v
txt/coq_projects/distributed-reference-counting/machine2/invariant6.v
txt/coq_projects/distributed-reference-counting/machine2/comm.v
txt/coq_projects/distributed-reference-counting/machine2/invariant2.v
txt/coq_projects/distributed-reference-counting/machine2/invariant8.v
txt/coq_projects/distributed-reference-counting/machine2/invariant5.v
txt/coq_projects/distributed-reference-counting/machine3/invariant1.v
txt/coq_projects/distributed-reference-counting/machine3/cardinal.v
txt/coq_projects/distributed-reference-counting/machine3/invariant3.v
txt/coq_projects/distributed-reference-counting/machine3/still_to_prove.v
txt/coq_projects/distributed

txt/coq_projects/verdi-raft/raft-proofs/AllEntriesLeaderSublogProof.v
txt/coq_projects/verdi-raft/raft-proofs/CausalOrderPreservedProof.v
txt/coq_projects/verdi-raft/raft-proofs/AllEntriesLogProof.v
txt/coq_projects/verdi-raft/raft-proofs/SortedProof.v
txt/coq_projects/verdi-raft/raft-proofs/MatchIndexLeaderProof.v
txt/coq_projects/verdi-raft/raft-proofs/RaftRefinementProof.v
txt/coq_projects/verdi-raft/raft-proofs/PrefixWithinTermProof.v
txt/coq_projects/verdi-raft/raft-proofs/LeadersHaveLeaderLogsStrongProof.v
txt/coq_projects/verdi-raft/raft-proofs/LeadersHaveLeaderLogsProof.v
txt/coq_projects/verdi-raft/raft-proofs/StateMachineSafetyPrimeProof.v
txt/coq_projects/verdi-raft/raft-proofs/TermsAndIndicesFromOneLogProof.v
txt/coq_projects/verdi-raft/raft-proofs/RequestVoteMaxIndexMaxTermProof.v
txt/coq_projects/verdi-raft/raft-proofs/OneLeaderLogPerTermProof.v
txt/coq_projects/verdi-raft/raft-proofs/VotesLeCurrentTermProof.v
txt/coq_projects/verdi-raft/raft-proofs/LeaderLogsCandidateEnt

txt/coq_projects/VST/concurrency/lksize.v
txt/coq_projects/VST/concurrency/memory_lemmas.v
txt/coq_projects/VST/concurrency/main.v
txt/coq_projects/VST/concurrency/semax_conc_pred.v
txt/coq_projects/VST/concurrency/memsem_lemmas.v
txt/coq_projects/VST/concurrency/semax_conc.v
txt/coq_projects/VST/concurrency/shim/Clight_core.v
txt/coq_projects/VST/concurrency/juicy/resource_decay_lemmas.v
txt/coq_projects/VST/concurrency/juicy/sync_preds.v
txt/coq_projects/VST/concurrency/juicy/semax_preservation_jspec.v
txt/coq_projects/VST/concurrency/juicy/semax_progress.v
txt/coq_projects/VST/concurrency/juicy/semax_simlemmas.v
txt/coq_projects/VST/concurrency/juicy/erasure_proof.v
txt/coq_projects/VST/concurrency/juicy/Clight_safety.v
txt/coq_projects/VST/concurrency/juicy/cl_step_lemmas.v
txt/coq_projects/VST/concurrency/juicy/JuicyMachineModule.v
txt/coq_projects/VST/concurrency/juicy/semax_to_juicy_machine.v
txt/coq_projects/VST/concurrency/juicy/resource_decay_join.v
txt/coq_projects/VST/concu

txt/coq_projects/VST/compcert_new/common/ExposedSmallstep.v
txt/coq_projects/VST/compcert_new/common/Values.v
txt/coq_projects/VST/compcert_new/common/Subtyping.v
txt/coq_projects/VST/compcert_new/common/Determinism.v
txt/coq_projects/VST/compcert_new/common/Errors.v
txt/coq_projects/VST/compcert_new/common/Separation.v
txt/coq_projects/VST/compcert_new/common/Behaviors.v
txt/coq_projects/VST/aes/aes.v
txt/coq_projects/VST/aes/verif_setkey_enc_LL_loop_body.v
txt/coq_projects/VST/aes/api_specs.v
txt/coq_projects/VST/aes/verif_encryption_LL_loop_body.v
txt/coq_projects/VST/aes/verif_encryption_LL_after_loop.v
txt/coq_projects/VST/aes/aes_encryption_loop_body.v
txt/coq_projects/VST/aes/verif_encryption_LL.v
txt/coq_projects/VST/aes/partially_filled.v
txt/coq_projects/VST/aes/spec_encryption_LL.v
txt/coq_projects/VST/aes/verif_gen_tables_LL.v
txt/coq_projects/VST/aes/equiv_encryption.v
txt/coq_projects/VST/aes/list_lemmas.v
txt/coq_projects/VST/aes/spec_utils_LL.v
txt/coq_projects/VST/aes/

In [155]:
paths = [str(x) for x in Path('txt').glob('coq_projects/**/*.v')]

Create Custom Tokenizer

In [156]:
tokenizer = Tokenizer(BPE())

All we want is to convert all special characters to Utf-8. NFKC is one of 4-ish major conversion schemes.

In [157]:
tokenizer.normalizer = Sequence([
    NFKC()
])
tokenizer.pre_tokenizer = ByteLevel()
tokenizer.decoder = ByteLevelDecoder()

Train the tokenizer (Byte Encoding)

In [172]:
trainer = BpeTrainer(vocab_size=25000, show_progress=True)
tokenizer.train(trainer, paths)

Test the tokenizer

In [173]:
with open(paths[-1], "r") as f:
    test = f.read()
    print("Input: {}".format(test))
    encoding = tokenizer.encode(test)
    print("Encoded: {}".format(encoding.tokens))
    decoding = tokenizer.decode(encoding.ids)
    print("Decoded: {}".format(decoding))

Input: (* This program is free software; you can redistribute it and/or      *)
(* modify it under the terms of the GNU Lesser General Public License *)
(* as published by the Free Software Foundation; either version 2.1   *)
(* of the License, or (at your option) any later version.             *)
(*                                                                    *)
(* This program is distributed in the hope that it will be useful,    *)
(* but WITHOUT ANY WARRANTY; without even the implied warranty of     *)
(* MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the      *)
(* GNU General Public License for more details.                       *)
(*                                                                    *)
(* You should have received a copy of the GNU Lesser General Public   *)
(* License along with this program; if not, write to the Free         *)
(* Software Foundation, Inc., 51 Franklin St, Fifth Floor, Boston, MA *)
(* 02110-1301 USA                           

In [25]:
model_ckpt = 'distilgpt2'

In [20]:
config = AutoConfig.from_pretrained(model_ckpt)

In [23]:
model = AutoModel.from_pretrained(model_ckpt)

Some weights of GPT2Model were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
